# Spot CNN Obstacle Detection — Colab Test Notebook

This notebook tests the CNN obstacle avoidance environment on Google Colab.

**Steps:**
1. Install dependencies & configure headless MuJoCo
2. Smoke test: environment loads and steps
3. Verify CNN feature extractor builds
4. Short training run (sanity check)
5. Visualise camera output from the robot

## 1. Install Dependencies & Setup Headless Rendering

In [ ]:
# Install system-level EGL libs for headless MuJoCo rendering
!apt-get update -qq
!apt-get install -y -qq libegl1-mesa-dev libgl1-mesa-dev libgles2-mesa-dev libglfw3-dev > /dev/null 2>&1

# Install Python packages
!pip install -q gymnasium>=0.29.0 mujoco>=3.0.0 numpy>=1.24.0
!pip install -q "stable-baselines3[extra]>=2.2.0" tensorboard>=2.14.0
!pip install -q torch>=2.0.0 torchvision>=0.15.0
!pip install -q matplotlib>=3.7.0 imageio>=2.31.0

print("\nDependencies installed!")

In [ ]:
# Configure MuJoCo to use EGL for offscreen rendering (no display needed)
import os
os.environ["MUJOCO_GL"] = "egl"

# Verify MuJoCo + EGL works
import mujoco
print(f"MuJoCo version: {mujoco.__version__}")

import gymnasium as gym
import numpy as np
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print("\nAll imports OK!")

## 2. Upload / Clone Project Files

**Option A** — Clone the repo (recommended):

In [ ]:
# Option A: Clone from GitHub (update URL to your repo)
# !git clone https://github.com/Raj9408612613/claude_code.git
# %cd claude_code/files

# Option B: If you uploaded the files manually to Colab, set the path:
# %cd /content/files

# For now, let's assume files are in the current directory.
# Uncomment one of the options above and run this cell.
print("Set your working directory above, then re-run this cell.")

In [ ]:
# Verify project files are present
required_files = [
    "spot_scene.xml",
    "spot_obstacle_env.py",
    "cnn_policy.py",
    "train_obstacle.py",
    "config.py",
]
for f in required_files:
    exists = os.path.exists(f)
    status = "OK" if exists else "MISSING"
    print(f"  [{status}] {f}")
    if not exists:
        raise FileNotFoundError(f"{f} not found — check your working directory (pwd: {os.getcwd()})")

print("\nAll files present!")

## 3. Smoke Test — Environment Loads and Steps

In [ ]:
from spot_obstacle_env import SpotObstacleEnv

# Create environment (no display needed — rendering is offscreen via EGL)
env = SpotObstacleEnv(render_mode=None, camera_width=64, camera_height=64, n_obstacles_active=5)
obs, info = env.reset(seed=42)

print("Observation space:", env.observation_space)
print("Action space:    ", env.action_space)
print()
print(f"Image shape:          {obs['image'].shape}  dtype={obs['image'].dtype}")
print(f"Proprioception shape: {obs['proprioception'].shape}  dtype={obs['proprioception'].dtype}")
print(f"Info: {info}")
print()

# Run 20 random steps
for i in range(20):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Step {i:2d}: reward={reward:+.3f}  min_obs_dist={info['min_obstacle_dist']:.2f}  "
          f"collisions={info['obstacle_collisions']}")
    if terminated or truncated:
        print("         -> Episode ended, resetting.")
        obs, info = env.reset()

env.close()
print("\nSmoke test PASSED!")

## 4. Visualise Camera Output

Show what the robot's front camera actually sees — obstacles should appear as red shapes.

In [ ]:
import matplotlib.pyplot as plt

env = SpotObstacleEnv(camera_width=128, camera_height=128, n_obstacles_active=5)
obs, info = env.reset(seed=7)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
fig.suptitle("Front Camera View (4 random resets)", fontsize=14)

for idx in range(4):
    obs, info = env.reset(seed=idx * 10)
    # Take a few steps so the robot is moving
    for _ in range(5):
        obs, _, _, _, info = env.step(env.action_space.sample())
    
    axes[idx].imshow(obs["image"])
    axes[idx].set_title(f"Seed {idx*10}\ndist_to_goal={info['distance_to_goal']:.1f}m")
    axes[idx].axis("off")

plt.tight_layout()
plt.show()
env.close()
print("Camera visualisation complete!")

## 5. Verify CNN Feature Extractor Builds

In [ ]:
from cnn_policy import SpotCNNExtractor
from stable_baselines3 import PPO
import torch as th

env = SpotObstacleEnv(camera_width=64, camera_height=64)

policy_kwargs = dict(
    features_extractor_class=SpotCNNExtractor,
    features_extractor_kwargs=dict(cnn_output_dim=128, proprio_hidden_dim=64),
    net_arch=dict(pi=[256, 128], vf=[256, 128]),
    activation_fn=th.nn.ReLU,
)

model = PPO("MultiInputPolicy", env, policy_kwargs=policy_kwargs, verbose=0)

print("PPO + CNN model created successfully!")
print()
print("Feature extractor:")
print(model.policy.features_extractor)
print()

# Count parameters
total_params = sum(p.numel() for p in model.policy.parameters())
trainable_params = sum(p.numel() for p in model.policy.parameters() if p.requires_grad)
print(f"Total parameters:     {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

env.close()
print("\nCNN extractor test PASSED!")

## 6. Short Training Run (Sanity Check)

Run a quick 5k-step training to verify the full loop works (rollouts, gradient updates, logging).

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor

def make_env(seed=0):
    def _init():
        e = SpotObstacleEnv(camera_width=64, camera_height=64, n_obstacles_active=5)
        e.reset(seed=seed)
        return Monitor(e)
    return _init

train_env = DummyVecEnv([make_env(seed=0)])

policy_kwargs = dict(
    features_extractor_class=SpotCNNExtractor,
    features_extractor_kwargs=dict(cnn_output_dim=128, proprio_hidden_dim=64),
    net_arch=dict(pi=[256, 128], vf=[256, 128]),
    activation_fn=th.nn.ReLU,
)

model = PPO(
    "MultiInputPolicy",
    train_env,
    learning_rate=3e-4,
    n_steps=512,       # smaller for quick test
    batch_size=64,
    n_epochs=4,        # fewer epochs for quick test
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.2,
    ent_coef=0.01,
    vf_coef=0.5,
    max_grad_norm=0.5,
    policy_kwargs=policy_kwargs,
    verbose=1,
)

print("Training for 5,000 steps (quick sanity check)...")
model.learn(total_timesteps=5_000, progress_bar=True)

train_env.close()
print("\nShort training run PASSED!")

## 7. Full Training (Optional)

Launch a proper training run. On a Colab T4 GPU, ~500k steps may take 1-2 hours.
Adjust `--timesteps` based on your available runtime.

In [ ]:
# Uncomment to run full training:
# !python train_obstacle.py --timesteps 500000 --n_envs 1 --camera_size 64 --n_obstacles 5

print("Uncomment the line above and run this cell to start full training.")
print("Monitor with TensorBoard:")
print("  %load_ext tensorboard")
print("  %tensorboard --logdir spot_obstacle_logs")

In [ ]:
# Load TensorBoard inline (run after training starts)
# %load_ext tensorboard
# %tensorboard --logdir spot_obstacle_logs

## Done!

If all cells above ran without errors, the CNN obstacle detection pipeline is working.

**Summary of what was tested:**
1. MuJoCo + EGL headless rendering
2. SpotObstacleEnv: Dict obs (image + proprioception), obstacle placement, collision detection
3. Camera output visualisation
4. SpotCNNExtractor: Nature-CNN + proprioception MLP
5. PPO training loop with MultiInputPolicy